

1. Pull all current sections enrollments that should be graded
2. Join with Platform Letter Grades
3. Convert letter grades to P/NP
4. Export CSV for SL review
5. Export TXT for student_grades import

In [1]:
import pandas as pd

from sps_automation import legacy as sps

conf = sps.load_config('./config/config.ini')

In [2]:
# create a database connection object for CA database
db_ca = sps.connect_to_ca_illuminate_db()

# TODO: create a database connection object for WA database
db_wa = sps.connect_to_wa_illuminate_db()

In [3]:
# query the CA database to pull all "gradable" enrollments.
query = """
SELECT
  local_student_id || ' ' || school_course_id AS stu_crs_lookup
  , sites.site_name
  , students.local_student_id
  , students.last_name
  , students.first_name
  , (matviews.ss_current.grade_level_id - 1) as current_grade_level
  , ssa.section_id
  , sections.local_section_id
  , departments.department_name
  , courses.school_course_id
  , courses.short_name
  , courses.variable_credits_high AS max_credits
  , date_part('days',CASE WHEN ssa.leave_date IS NULL THEN now() ELSE ssa.leave_date END - ssa.entry_date) AS days_enrolled
  , date_part('days',CASE WHEN ssa.leave_date IS NULL THEN now() ELSE ssa.leave_date END - ssa.entry_date) < 220 AS short_enrollment

FROM
  matviews.ss_current

LEFT JOIN section_student_aff as ssa on ssa.student_id = matviews.ss_current.student_id
LEFT JOIN students on ssa.student_id = students.student_id
LEFT JOIN sections ON ssa.section_id = sections.section_id
LEFT JOIN courses ON ssa.course_id = courses.course_id
LEFT JOIN sites on matviews.ss_current.site_id = sites.site_id
LEFT JOIN departments on courses.department_id = departments.department_id

WHERE
  matviews.ss_current.site_id NOT IN (9999998,9999999) AND
  courses.transcript_inclusion IS NOT FALSE AND
  courses.is_active IS TRUE AND
  courses.variable_credits_high  >= 0.5 AND
  (ssa.leave_date > now() OR ssa.leave_date IS NULL) AND
  ssa.entry_date > '2017-08-01'

GROUP BY
  sites.site_name
  , students.local_student_id
  , students.last_name
  , students.first_name
  , matviews.ss_current.grade_level_id
  , ssa.section_id
  , sections.local_section_id
  , departments.department_name
  , courses.school_course_id
  , courses.short_name
  , ssa.entry_date
  , ssa.leave_date
  , courses.variable_credits_high

ORDER BY
  site_name
  , grade_level_id
  , last_name
  , first_name
  , school_course_id
"""

In [4]:
#Load query results as "gradable sections"
gs_ca = pd.read_sql(query,db_ca['connection'])
gs_wa = pd.read_sql(query,db_wa['connection'])

#make sure 'local_student_id' is INT
gs_ca['local_student_id'] = pd.to_numeric(gs_ca['local_student_id'])
gs_wa['local_student_id'] = pd.to_numeric(gs_wa['local_student_id'])

In [5]:
#join CA and WA into a single df
gs_all = pd.concat ([gs_ca,gs_wa])

#export to CSV
gs_all.to_csv('../data/graded_sections_export.csv')

In [11]:
# assert that we have the same number of rows after combining as before - Note that shape[0]
#  gives us the first number that shape holds - the rows
assert gs_all.shape[0] == gs_ca.shape[0] + gs_wa.shape[0]

#assert that we have the same number of columns after combining as before
assert gs_all.shape[1] == gs_ca.shape[1] 
assert gs_all.shape[1] == gs_wa.shape[1]

In [16]:
#TODO: Read in the platform grades CSV to a dataframe called pl_grades and 

pl_dump = pd.read_csv('../data/platformgrades.csv', encoding='latin-1')

#create a lookup index by combining local id with course ID. Should match first column of queried sections
#pl_dump['stu_crs_lookup'] = pl_dump['STUDENT_ID'].map(str) + ' ' + pl_dump['COURSE_NUMBER']

#drop duplicate records
pl_nodups = pl_dump.drop_duplicates()



#merge queried sections with platform courses

merged_grades = pd.merge(gs_all, pl_nodups, on = 'stu_crs_lookup', how = 'left',suffixes=('_ill','_pltfrm'))

merged_grades.head(10)

,stu_crs_lookup,site_name,local_student_id,last_name,first_name,current_grade_level,section_id,local_section_id,department_name,school_course_id,...,POWER_OUT_OF,RAW_COG_SKILL_SCORE,RAW_CONCEPT_SCORE,SECTION_NAME,SECTION_SITE_NAME,SIS_SECTION_ID,STUDENT_FIRST_NAME,STUDENT_ID,STUDENT_LAST_NAME,SUBJECT_CATEGORY
0,21266 A100,Everest Public High School,21266,Abild-Pedersen,Jonas,9,75781,75781,History,A100,...,9.0,4.288462,NaN,World Studies I - Buller - 13.T3A,Everest Public High School,75781_2017-08-15,Jonas,21266.0,Abild-Pedersen,core
1,21266 B100,Everest Public High School,21266,Abild-Pedersen,Jonas,9,75750,75750,English,B100,...,14.0,4.533333,NaN,English 9 - Shamaeva - 01.M1A,Everest Public High School,75750_2017-08-15,Jonas,21266.0,Abild-Pedersen,core
2,21266 C120,Everest Public High School,21266,Abild-Pedersen,Jonas,9,75797,75797,Math,C120,...,10.0,5.333333,4.31,Mathematics I - Horovitz - 11.T2A,Everest Public High School,75797_2017-08-15,Jonas,21266.0,Abild-Pedersen,core
3,21266 D100,Everest Public High School,21266,Abild-Pedersen,Jonas,9,75840,75840,Science,D100,...,9.0,3.523810,NaN,Biology - Sacksteder - 05.M3A,Everest Public High School,75840_2017-08-15,Jonas,21266.0,Abild-Pedersen,core
4,21266 E100,Everest Public High School,21266,Abild-Pedersen,Jonas,9,75775,75775,Spanish,E100,...,12.0,NaN,NaN,Spanish 1 - Meier - 03.M2A,Everest Public High School,75775_2017-08-15,Jonas,21266.0,Abild-Pedersen,core
5,21266 F511,Everest Public High School,21266,Abild-Pedersen,Jonas,9,81990,2018EVERF511PMEsp,Visual and Performing Arts,F511,...,0.0,NaN,NaN,Intro to Video Production - Esparza-Johnson - ...,Everest Public High School,81990_2017-08-15,Jonas,21266.0,Abild-Pedersen,esoteric
6,21266 I1999,Everest Public High School,21266,Abild-Pedersen,Jonas,9,83165,83165,Intersession,I1999,...,0.0,NaN,NaN,Communications - Redlin - AM Expeditions EV,Everest Public High School,83165_2017-08-15,Jonas,21266.0,Abild-Pedersen,esoteric
7,21219 A100,Everest Public High School,21219,Aguilar-Sosa,Oscar,9,75781,75781,History,A100,...,9.0,3.653846,NaN,World Studies I - Buller - 13.T3A,Everest Public High School,75781_2017-08-15,Oscar,21219.0,Aguilar-Sosa,core
8,21219 B100,Everest Public High School,21219,Aguilar-Sosa,Oscar,9,75820,75820,English,B100,...,14.0,NaN,NaN,English 9 - Shamaeva - 05.M3A,Everest Public High School,75820_2017-08-15,Oscar,21219.0,Aguilar-Sosa,core
9,21219 C120,Everest Public High School,21219,Aguilar-Sosa,Oscar,9,75850,75850,Math,C120,...,10.0,5.166667,2.81,Mathematics I - Horovitz - 03.M2A,Everest Public High School,75850_2017-08-15,Oscar,21219.0,Aguilar-Sosa,core


In [18]:
#export csv
merged_grades.to_csv('../data/merged_grades_export.csv')

#assert gs_all.shape[1] == merged_grades.shape[1]